<a href="https://colab.research.google.com/github/Vanpersact/hello-world/blob/main/Filtro_Adjetivos_Frequencias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Filtro de **Adjetivos** com Tolerância a Erros (PT-BR)  
Cria uma tabela somente com **adjetivos** e seus quantitativos, a partir da lista de frequências de palavras.

**Como usar**  
- **Opção A — Arquivo**: informe `ARQUIVO = "minha_lista.xlsx"` (ou `.csv`) com colunas `Palavra` e `Frequência` (ou similares).  
- **Opção B — Colar texto**: cole a lista na variável `COLAR_LISTA` (duas colunas, separadas por TAB ou por múltiplos espaços).

O notebook:
1. Normaliza (minúsculas, remove acentos, reduz repetições: `lotadoooo` → `lotadoo`);  
2. Aplica **regras morfológicas** e um **léxico semente** de adjetivos;  
3. Faz **fuzzy matching** leve (distância de edição) para capturar erros simples;  
4. Agrupa por **lema** e exporta **Excel** com:
   - `01_adjetivos_detalhado` (todas as formas encontradas),
   - `02_adjetivos_agrupado` (somado por lema normalizado).

> O método é **heurístico** (sem modelo pesado). Bom para começar; posso evoluir depois com POS-tagging.


In [2]:

# (Se estiver em ambiente novo como Colab, descomente para instalar dependências leves)
!pip install pandas openpyxl unidecode


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 14.8 MB/s eta 0:00:00


In [3]:
import re, io, sys
import pandas as pd
from unidecode import unidecode
from pathlib import Path
from typing import List, Tuple

In [4]:
# ====================== CONFIGURAÇÃO ======================
# Informe um arquivo OU cole o texto bruto.
ARQUIVO = "Adjetivos_por_Linha.xlsx"
COLAR_LISTA = """
# Cole aqui (opcional): duas colunas — Palavra e Frequência (TAB ou múltiplos espaços)
# Exemplo:
# absurdo	244
# atrasada	135
# lotado	383
"""

# Nomes possíveis das colunas no arquivo
COLS_PALAVRA = {"PALAVRA"}
COLS_FREQ    = {"FREQUENCIA"}

# ==========================================================



In [5]:
def ler_tabela():
    if ARQUIVO:
        p = Path(ARQUIVO)
        if not p.exists():
            raise FileNotFoundError(f"Arquivo não encontrado: {p.resolve()}")
        if p.suffix.lower() in [".xlsx", ".xls"]:
            df = pd.read_excel(p)
        elif p.suffix.lower() == ".csv":
            # tenta separadores comuns
            try:
                df = pd.read_csv(p)
            except Exception:
                df = pd.read_csv(p, sep=";")
        else:
            raise ValueError("Forneça .xlsx/.xls ou .csv")
        cols_lower = {c.lower(): c for c in df.columns}
        col_pal = None
        col_freq = None
        for k,v in cols_lower.items():
            if k in COLS_PALAVRA and col_pal is None:
                col_pal = v
            if k in COLS_FREQ and col_freq is None:
                col_freq = v
        if col_pal is None or col_freq is None:
            # tenta a primeira e segunda coluna como fallback
            col_pal = list(df.columns)[0]
            col_freq = list(df.columns)[1]
        df = df[[col_pal, col_freq]].copy()
        df.columns = ["Palavra","Frequencia"]
        return df
    else:
        # Parse do texto colado
        bruto = COLAR_LISTA.strip().splitlines()
        dados = []
        for ln in bruto:
            ln = ln.strip()
            if not ln or ln.startswith("#"):
                continue
            # tenta TAB, senão múltiplos espaços
            if "\t" in ln:
                partes = ln.split("\t")
            else:
                partes = re.split(r"\s{2,}", ln)  # 2+ espaços
                if len(partes) == 1:
                    partes = re.split(r"\s+", ln, maxsplit=1)
            if len(partes) >= 2:
                w, f = partes[0].strip(), partes[1].strip()
                dados.append((w, f))
        df = pd.DataFrame(dados, columns=["Palavra","Frequencia"])
        return df


In [6]:
df_raw = ler_tabela()
print("Linhas lidas:", len(df_raw))
df_raw.head(10)

Linhas lidas: 7592


,Palavra,Frequencia
0,00,258
1,0,7
2,000,5
3,1,523
4,01,81
5,2,389
6,02,89
7,3,314
8,03,72
9,4,317


In [7]:
# --------- Normalização ---------
def normalizar_palavra(w: str) -> str:
    w0 = str(w).strip()
    w0 = w0.replace("’","'").replace("`","'")
    w0 = unidecode(w0.lower())
    # reduz repetições exageradas de letras: kkkkkk -> kk, lotadoooo -> lotadoo
    w0 = re.sub(r"(.)\1{2,}", r"\1\1", w0)
    # remove pontuações nas pontas
    w0 = re.sub(r"^[^a-z]+|[^a-z]+$", "", w0)
    return w0

def eh_palavra_valida(w: str) -> bool:
    if not w: return False
    # rejeita tokens com dígitos
    if re.search(r"\d", w): return False
    # tamanho mínimo
    if len(w) < 3: return False
    return True




In [8]:
df = df_raw.copy()
df["Frequencia"] = pd.to_numeric(df["Frequencia"].astype(str).str.replace(".","").str.replace(",","."), errors="coerce")
df = df.dropna(subset=["Frequencia"])
df["Frequencia"] = df["Frequencia"].astype(int)

df["pal_norm"] = df["Palavra"].apply(normalizar_palavra)
df = df[df["pal_norm"].apply(eh_palavra_valida)].copy()
df = df.groupby(["pal_norm"], as_index=False)["Frequencia"].sum()
print("Após normalização:", len(df), "tokens únicos")
df.head(10)

Após normalização: 6486 tokens únicos


,pal_norm,Frequencia
0,aba,2
1,abafado,33
2,abaixo,9
3,abalada,7
4,abanando,2
5,abandonada,9
6,abandonado,5
7,abandono,3
8,abandonou,3
9,abarrotadas,5


In [9]:
# --------- Dicionário semente de adjetivos comuns (PT-BR) ---------
ADJ_SEMENTE = {
    # polaridade / qualidade geral
    "bom","boa","otimo","otima","excelente","maravilhoso","maravilhosa","perfeito","perfeita",
    "ruim","pessimo","pessima","horrivel","terrivel","mediano","mediana","regular",
    "absurdo","absurda","inaceitavel","inadmissivel","incrivel","deploravel","insuportavel",
    "impossivel","desagradavel","agradavel","necessario","desnecessario","essencial",
    "complicado","complicada","simples","facil","dificil","rapido","rapida","lento","lenta",
    "caro","cara","barato","barata","cheio","cheia","vazio","vazia","sujo","suja","limpo","limpa",
    "escuro","escura","claro","clara","seguro","segura","perigoso","perigosa",
    "eficiente","ineficiente","competente","incompetente",
    "confortavel","desconfortavel","normal","anormal",
    "frio","fria","quente","gelado","gelada","lotado","lotada","lotados","lotadas",
    "atrasado","atrasada","adiantado","adiantada",
    "barulhento","barulhenta","silencioso","silenciosa",
    "higienico","higienica","insalubre","salubre",
    "legal","ilegal","justo","injusto","honesto","desonesto",
    "educado","educada","gentil","grossa","grosso","nojento","nojenta",
    "grande","pequeno","alto","baixa","baixo","tenso","tensa","calmo","calma",
    "preciso","impreciso","correto","incorreto","verdadeiro","falso",
    "otimizado","otimizada","minimo","maximo","minima","maxima",
    "horrendo","horrenda","horroroso","horrorosa","terrivel",
    "lindo","linda","bonito","bonita","feio","feia"
}

# Reduções/lematizações simples (agrupar formas flexionadas no mesmo lema)
LEMA_EQUIV = {
    "boas":"boa", "bons":"bom", "melhor":"bom", "melhores":"bom",
    "pior":"pessimo", "piores":"pessimo",
    "otimos":"otimo","otimas":"otima","excelentes":"excelente",
    "maravilhosos":"maravilhoso","maravilhosas":"maravilhosa",
    "absurdos":"absurdo","absurdas":"absurda","absurso":"absurdo",
    "inaceitaveis":"inaceitavel","inadmissiveis":"inadmissivel",
    "rapidos":"rapido","rapidas":"rapida","lentos":"lento","lentas":"lenta",
    "caros":"caro","caras":"cara","baratos":"barato","baratas":"barata",
    "cheios":"cheio","cheias":"cheia","vazios":"vazio","vazias":"vazia",
    "sujos":"sujo","sujas":"suja","limpos":"limpo","limpas":"limpa",
    "claros":"claro","claras":"clara","escuros":"escuro","escuras":"escura",
    "seguros":"seguro","seguras":"segura","perigosos":"perigoso","perigosas":"perigosa",
    "eficientes":"eficiente","competentes":"competente","incompetentes":"incompetente",
    "confortaveis":"confortavel","desconfortaveis":"desconfortavel",
    "normais":"normal","anormais":"anormal",
    "frios":"frio","frias":"fria","quentes":"quente","gelados":"gelado","geladas":"gelada",
    "lotados":"lotado","lotadas":"lotada","lotaderrimo":"lotado","lotadissimo":"lotado","lotaderrima":"lotada","lotadissima":"lotada",
    "atrasados":"atrasado","atrasadas":"atrasada","adiantados":"adiantado","adiantadas":"adiantada",
    "barulhentos":"barulhento","barulhentas":"barulhenta",
    "insalubres":"insalubre",
    "justos":"justo","injustos":"injusto","honestos":"honesto","desonestos":"desonesto",
    "educados":"educado","educadas":"educada","gentis":"gentil","grossos":"grosso","grossas":"grossa",
    "nojentos":"nojento","nojentas":"nojenta",
    "grandes":"grande","pequenos":"pequeno","pequenas":"pequeno",
    "altos":"alto","altas":"alto","baixos":"baixo","baixas":"baixo",
    "tensos":"tenso","tensas":"tensa","calmos":"calmo","calmas":"calma",
    "precisos":"preciso","imprecisos":"impreciso","corretos":"correto","incorretos":"incorreto",
    "verdadeiros":"verdadeiro","falsos":"falso",
    "minimos":"minimo","maximos":"maximo","minimas":"minima","maximas":"maxima",
    "horrendos":"horrendo","horrendas":"horrenda","horrorosos":"horroroso","horrorosas":"horrorosa",
    "terriveis":"terrivel","feios":"feio","feias":"feia","bonitos":"bonito","bonitas":"bonita",
}

def lema_normal(w: str) -> str:
    return LEMA_EQUIV.get(w, w)

# --------- Distância de edição simples (Levenshtein) ---------
def lev_dist(a: str, b: str) -> int:
    # otimização simples
    if a == b: return 0
    if abs(len(a)-len(b)) > 2:  # atalho: longe demais
        return 99
    m, n = len(a), len(b)
    dp = list(range(n+1))
    for i, ca in enumerate(a, 1):
        prev, dp[0] = dp[0], i
        for j, cb in enumerate(b, 1):
            cur = min(
                dp[j] + 1,
                dp[j-1] + 1,
                prev + (ca != cb)
            )
            prev, dp[j] = dp[j], cur
    return dp[n]

# --------- Heurísticas de "parece adjetivo" ---------
SUFIXOS_ADJ = (
    "vel","veis","ivel","iveis","avel","aveis",
    "oso","osa","osos","osas",
    "ado","ada","ados","adas",
    "ido","ida","idos","idas",
    "ento","enta","entos","entas",
    "ante","antes","ente","entes",
    "al","ais","ar","ares","ario","aria","arios","arias",
    "udo","uda","udos","udas",
)
def parece_adjetivo(w: str) -> bool:
    # regras rápidas
    if w in ADJ_SEMENTE: return True
    if any(w.endswith(s) for s in SUFIXOS_ADJ):
        return True
    return False

# --------- Mapeamento por fuzzy p/ erros simples ---------
def fuzzy_mapeia_adjetivo(w: str) -> tuple[str, str]:
    """
    Se 'w' for (ou parecer) adjetivo, retorna (lema, motivo). Caso contrário, ("","").
    """
    w0 = lema_normal(w)
    if w0 in ADJ_SEMENTE:
        return w0, "lexico"
    if parece_adjetivo(w0):
        return w0, "regra"
    # tenta aproximar aos lemas do dicionário semente
    melhor = None
    melhor_d = 99
    for cand in ADJ_SEMENTE:
        d = lev_dist(w0, cand)
        if d < melhor_d:
            melhor_d, melhor = d, cand
    # tolerâncias: 1 para curtas (<=5), 2 para outras
    tol = 1 if len(w0) <= 5 else 2
    if melhor_d <= tol:
        return melhor, f"fuzzy(d={melhor_d})"
    return "", ""

# Aplica
saida_linhas = []
for _, row in df.iterrows():
    w = row["pal_norm"]
    freq = int(row["Frequencia"])
    lema, motivo = fuzzy_mapeia_adjetivo(w)
    if lema:
        saida_linhas.append({
            "adjetivo_encontrado": w,
            "frequencia": freq,
            "lema_normalizado": lema,
            "criterio": motivo
        })

df_adj = pd.DataFrame(saida_linhas)
df_adj = df_adj.sort_values(["lema_normalizado","frequencia"], ascending=[True, False]).reset_index(drop=True)
print("Adjetivos detectados:", len(df_adj))
df_adj.head(20)


Adjetivos detectados: 1925


,adjetivo_encontrado,frequencia,lema_normalizado,criterio
0,abafado,33,abafado,regra
1,abalada,7,abalada,regra
2,abandonada,9,abandonada,regra
3,abandonado,5,abandonado,regra
4,abarrotadas,5,abarrotadas,regra
5,abarrotado,10,abarrotado,regra
6,abarrotados,2,abarrotados,regra
7,abastecidas,3,abastecidas,regra
8,abencoado,2,abencoado,regra
9,abonar,5,abonar,regra


In [10]:

# --------- Agrupa por lema ---------
df_grp = df_adj.groupby("lema_normalizado", as_index=False)["frequencia"].sum()
df_grp = df_grp.sort_values("frequencia", ascending=False).reset_index(drop=True)
df_grp.head(20)


,lema_normalizado,frequencia
0,bom,6040
1,cara,3991
2,linda,3476
3,sentido,1928
4,mais,1676
5,caro,1445
6,boa,1101
7,horario,1010
8,nada,945
9,parado,892


In [11]:

# --------- Exporta Excel ---------
from datetime import datetime
import pandas as pd
from pathlib import Path

out_path = Path("adjetivos_filtrados.xlsx")
with pd.ExcelWriter(out_path, engine="openpyxl") as xls:
    df_adj.to_excel(xls, sheet_name="01_adjetivos_detalhado", index=False)
    df_grp.to_excel(xls, sheet_name="02_adjetivos_agrupado", index=False)
print("Arquivo gerado:", out_path.resolve())


Arquivo gerado: /content/adjetivos_filtrados.xlsx
